In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.shape

(404290, 6)

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


##### Creating a smaller dataset.

In [5]:
new_df = df.sample(30000)

In [6]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [7]:
new_df.duplicated().sum()

0

In [8]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
262444,Is Hinduism monotheistic or polytheistic?,Do Hindus believe in one God?
326480,Will upgrading to the 2016 MacBook Pro be wort...,Is the 2016 MacBook Pro still worth it (as opp...
163714,Why is my pinky finger numb?,Why is my pinky finger itchy?
274962,What are some ideas for a web development proj...,Has anyone got some ideas for a project while ...
305604,What are your top 5 movies for 2016?,What are your Top 5 movies?


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

##### Now , we will convert them to pandas dataframe and concat both.

In [10]:
temp_df1=pd.DataFrame(q1_arr,index= ques_df.index)
temp_df2=pd.DataFrame(q2_arr, index= ques_df.index)
temp_df=pd.concat([temp_df1,temp_df2],axis=1)
temp_df.shape


(30000, 6000)

In [11]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
262444,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
326480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163714,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
274962,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
305604,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120460,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
121484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
112817,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
373438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [13]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
262444,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
326480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
163714,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
274962,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
305604,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


##### Now , we will perform train_test_split.

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=42)

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7431666666666666

In [17]:
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/24/ec/ad387100fa3cc2b9b81af0829b5ecfe75ec5bb19dd7c19d4fea06fb81802/xgboost-2.0.3-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 187.9 kB/s eta 0:08:51
   ---------------------------------------- 0.0/99.8 MB 196.9 kB/s eta 0:08:27
   ---------------------------------------- 0.1/99.8 MB 302.7 kB/s eta 0:05:30
   ---------------------------------------- 0.2/99.8 MB 573.4 kB/s eta 0:02:54
   ---------------------------------------- 0.4/99.8 MB 1.2 MB/s eta 0:01:23
   ---------------------------------------- 0.8/99.8 MB 2.3 MB/s eta 0:00:44
   ---------------------------------------- 1

In [18]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7198333333333333